In [1]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import csv
import os

import keras
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Cropping2D, AveragePooling2D, Convolution2D, Dropout, ELU

print('imported')

Using TensorFlow backend.


imported


In [2]:
#Settings
split=0.15        #Split overall image data into training and validation set
resize_factor=1   #resize the images by this factor *actually has no effect anymore, as 2x2-average-pooling is used as resizing by half
crop_top=65       #crop uninteresting area, like far away objects and skyline
crop_bot=15       #crop uninteresting area that is too near or the hood
dropout1=0.02     #dropout-rate between layer 1 and 2
dropout2=0.03     #dropout-rate between layer 2 and 3
dropout3=0.20     #dropout-rate between layer 3 and 4
dropout4=0.45     #dropout-rate between layer 4 and 5
learning_rate=0.025
decay=1e-5
nesterov_momentum=0.85
batch=250         #Batch size of images that are computed at once
epochs=15         #Number of epochs the network is trained 

#resize function, because lambda need tf be imported at each call, else it causes an error
#def resize_image(images,factor,in_height,in_width):
#    from keras.backend import tf as ktf
#return ktf.image.resize_images(images, (int(in_height*factor), int(in_width*factor)))


samples = []
with open('M:/records p3/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
#Split data into training and validation sets
train_samples, validation_samples = train_test_split(samples, test_size=split)

def generator(samples, batch_size=batch): #As the image data is mirrored, twice the batch_size is loaded at the same time
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            for batch_sample in batch_samples:
                name = batch_sample[0]                 # 'M:/records p3/IMG/...' [0]for center camera
                image_data = cv2.imread(name)
                image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)# switch it to RGB, as drive.py sends RGB
                steering_angle = float(batch_sample[3])
                images.append(image_data)
                measurements.append(steering_angle)
                ##### mirror data vertically #######################
                image_data=np.fliplr(image_data)
                images.append(image_data)
                steering_angle=-steering_angle
                measurements.append(steering_angle)

            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch)
validation_generator = generator(validation_samples, batch_size=batch)

in_shape=((cv2.imread(samples[20][0])).shape[0],(cv2.imread(samples[20][0])).shape[1],(cv2.imread(samples[20][0])).shape[2])

####print informations for documentation####
print('Images normalized to [-1,+1]')
 #print(in_shape, 'original image dimensions, resized to', (int(in_shape[0]*resize_factor), int(in_shape[1]*resize_factor)))
print(in_shape, 'image dimensions, cropped by ', crop_top, 'px from top and ', crop_bot, 'px from bot')
print(int(len(samples)*2*(1-split)),'training images out of ',len(samples)*2, 'overall images [',split*100,'% for validation set]' )
print(batch, 'batch-size, ',learning_rate,'learning rate, ', decay,'decay, ', nesterov_momentum,'nesterov-momentum')
print('Dropout-Rates: Layer 1-2:',int(dropout1*100),'%, Layer 1-2:',int(dropout2*100),'%, Layer 1-2:',int(dropout3*100),'%, Layer 1-2:',int(dropout4*100),'%')
##############################
#Neural Network Architecture
##############################
model=Sequential()
model.add(Lambda(lambda X_train: ((X_train/127.5)-1), input_shape=in_shape)) #Normalization to [-1, ... ,+1]
model.add(Cropping2D(cropping=((crop_top,crop_bot), (0,0)), input_shape=in_shape)) #Cropping uninteresting image area
#model.add(AveragePooling2D(pool_size=(2, 2), strides=2, data_format='channels_first')) #Resizing the image by half according to: https://discussions.udacity.com/t/issues-resizing-images/228196/22
#model.add(Lambda(lambda X_train: resize_image(X_train, resize_factor, in_shape[0], in_shape[1]))) 

##CNN Architecture taken from (c) by Nvidia, implemented by (c) Udacity 
#model.add(Convolution2D(24,(5,5), strides=(2,2),activation="relu"))
#model.add(Dropout(dropout1))
#model.add(Convolution2D(36,(5,5), strides=(2,2),activation="relu"))
#model.add(Dropout(dropout2))
#model.add(Convolution2D(48,(5,5), strides=(2,2),activation="relu"))
#model.add(Dropout(dropout3))
#model.add(Convolution2D(64,(3,3), activation="relu"))
#model.add(Dropout(dropout4))
#model.add(Convolution2D(64,(3,3), activation="relu"))
#model.add(Flatten())
#model.add(Dense(100))
#model.add(Dense(50))
#model.add(Dense(1))

##VGG16 as mentioned in cheatsheet (https://github.com/commaai/research/blob/master/train_steering_model.py),
###with a few changes
model.add(Convolution2D(16,(8,8), strides=(4, 4), padding="same",activation="relu"))
model.output_shape
model.add(Dropout(dropout1))
model.add(Convolution2D(32,(5,5), strides=(2, 2), padding="same",activation="relu"))
model.output_shape
model.add(Dropout(dropout2))
model.add(Convolution2D(64,(5,5), strides=(2, 2), padding="same",activation="relu"))
model.add(Flatten())
model.add(Dropout(dropout3))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(dropout4))
model.add(ELU())
#model.add(Dense(250))
model.add(Dense(50))
model.add(Dense(1))

sgd = optimizers.SGD(lr=learning_rate, decay=decay, momentum=nesterov_momentum, nesterov=True)
model.compile(loss='mse', optimizer=sgd)
model.fit_generator(train_generator, steps_per_epoch=len(train_samples)/batch,epochs=epochs, validation_data=validation_generator, validation_steps=len(validation_samples)/batch)
model.save('model.h5')

Images normalized to [-1,+1]
(160, 320, 3) image dimensions, cropped by  65 px from top and  15 px from bot
47586 training images out of  55984 overall images [ 15.0 % for validation set]
250 batch-size,  0.025 learning rate,  1e-05 decay,  0.85 nesterov-momentum
Dropout-Rates: Layer 1-2: 2 %, Layer 1-2: 3 %, Layer 1-2: 20 %, Layer 1-2: 45 %
Epoch 1/15
42/95 [============>.................] - ETA: 290s - loss: 0.0738

KeyboardInterrupt: 

In [2]:
from keras.models import load_model
from keras.utils import plot_model

model=load_model('C:/Users/Reiners/Desktop/p3_driving/windows_sim/model.h5')

plot_model(model, to_file='modelvisualization.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

### First serious approach
- 18858x2 images, 15% validation-set-split
- normalized to [-1,+1], original sized
- Generator with batchsize=800
- Most basic NN architecture from lesson
---
    Epoch 1/5 loss: 40.2280 - val_loss: 11.4453
    Epoch 2/5 loss: 4.5891 - val_loss: 1.2830
    Epoch 3/5 loss: 0.9326 - val_loss: 0.6150
    Epoch 4/5 loss: 0.3673 - val_loss: 0.2114
    Epoch 5/5 loss: 0.1359 - val_loss: 0.1609
No overfitting and still reducing loss, driving ok, but drives offroad and keeps driving next to the curbs offroad

Images normalized to [-1,+1]
(160, 320, 3) original image dimensions, resized to (160, 320)
Image cropped by  65 px from top and  15 px from bot
Dropout-Rates: Layer 1-2: 1 %, Layer 1-2: 2 %, Layer 1-2: 30 %, Layer 1-2: 50 %
33644 training images out of  39582 overall images [ 15.0 % for validation set]
learning=0.008, decay=1e-6, nesterovmomentum=0.4
250 batch-size
Epoch 1/10
68/67 [==============================] - 61s - loss: 0.0364 - val_loss: 0.0364
[...]
Epoch 8/10
68/67 [==============================] - 54s - loss: 0.0262 - val_loss: 0.0279
Epoch 9/10
68/67 [==============================] - 54s - loss: 0.0262 - val_loss: 0.0276
Epoch 10/10
68/67 [==============================] - 54s - loss: 0.0258 - val_loss: 0.0274

leaves curve because of not tunrning enough

Images normalized to [-1,+1]
(160, 320, 3) original image dimensions, resized to (160, 320)
Image cropped by  65 px from top and  15 px from bot
Dropout-Rates: Layer 1-2: 2 %, Layer 1-2: 3 %, Layer 1-2: 18 %, Layer 1-2: 36 %
33644 training images out of  39582 overall images [ 15.0 % for validation set]
learning=0.015, decay=1e-6, nesterovmomentum=0.7
250 batch-size
Epoch 1/5
68/67 [==============================] - 60s - loss: 0.0323 - val_loss: 0.0252
Epoch 2/5
68/67 [==============================] - 55s - loss: 0.0270 - val_loss: 0.0239
Epoch 3/5
68/67 [==============================] - 58s - loss: 0.0260 - val_loss: 0.0231
Epoch 4/5
68/67 [==============================] - 63s - loss: 0.0253 - val_loss: 0.0225
Epoch 5/5
68/67 [==============================] - 62s - loss: 0.0247 - val_loss: 0.0220

till bridge ok, on bridge allways into wall

vgg16
Images normalized to [-1,+1]
(160, 320, 3) image dimensions, cropped by  65 px from top and  15 px from bot
37777 training images out of  44444 overall images [ 15.0 % for validation set]
250 batch-size,  0.015 learning rate,  1e-06 decay,  0.7 nesterov-momentum
Dropout-Rates: Layer 1-2: 2 %, Layer 1-2: 3 %, Layer 1-2: 18 %, Layer 1-2: 36 %
Epoch 1/5
76/75 [==============================] - 58s - loss: 0.0403 - val_loss: 0.0277
Epoch 2/5
76/75 [==============================] - 54s - loss: 0.0324 - val_loss: 0.0259
Epoch 3/5
76/75 [==============================] - 54s - loss: 0.0305 - val_loss: 0.0247
Epoch 4/5
76/75 [==============================] - 54s - loss: 0.0295 - val_loss: 0.0239
Epoch 5/5
76/75 [==============================] - 54s - loss: 0.0285 - val_loss: 0.0232

good, a bit too weak turning in corners, bridge perfect, tight corner approached with very low steering angle and drives off road

Images normalized to [-1,+1]
(160, 320, 3) image dimensions, cropped by  65 px from top and  15 px from bot
37777 training images out of  44444 overall images [ 15.0 % for validation set]
250 batch-size,  0.02 learning rate,  1e-05 decay,  0.85 nesterov-momentum
Dropout-Rates: Layer 1-2: 2 %, Layer 1-2: 3 %, Layer 1-2: 18 %, Layer 1-2: 36 %
Epoch 1/5
76/75 [==============================] - 171s - loss: 0.0358 - val_loss: 0.0273
Epoch 2/5
76/75 [==============================] - 158s - loss: 0.0287 - val_loss: 0.0249
Epoch 3/5
76/75 [==============================] - 60s - loss: 0.0270 - val_loss: 0.0238
Epoch 4/5
76/75 [==============================] - 59s - loss: 0.0259 - val_loss: 0.0229
Epoch 5/5
76/75 [==============================] - 57s - loss: 0.0251 - val_loss: 0.0225
better but still as before

Images normalized to [-1,+1]
(160, 320, 3) image dimensions, cropped by  65 px from top and  15 px from bot
47586 training images out of  55984 overall images [ 15.0 % for validation set]
250 batch-size,  0.025 learning rate,  1e-05 decay,  0.85 nesterov-momentum
Dropout-Rates: Layer 1-2: 2 %, Layer 1-2: 3 %, Layer 1-2: 20 %, Layer 1-2: 45 %
Epoch 1/8
96/95 [==============================] - 515s - loss: 0.0536 - val_loss: 0.0411
Epoch 2/8
96/95 [==============================] - 67s - loss: 0.0414 - val_loss: 0.0380
Epoch 3/8
96/95 [==============================] - 65s - loss: 0.0387 - val_loss: 0.0364
Epoch 4/8
96/95 [==============================] - 67s - loss: 0.0370 - val_loss: 0.0353
Epoch 5/8
96/95 [==============================] - 64s - loss: 0.0357 - val_loss: 0.0349
Epoch 6/8
96/95 [==============================] - 68s - loss: 0.0347 - val_loss: 0.0337
Epoch 7/8
96/95 [==============================] - 72s - loss: 0.0335 - val_loss: 0.0328
Epoch 8/8
96/95 [==============================] - 70s - loss: 0.0327 - val_loss: 0.0324

works reliable. a bit too outside of the corner but still without touching lane lines

Images normalized to [-1,+1]
(160, 320, 3) image dimensions, cropped by  65 px from top and  15 px from bot
47586 training images out of  55984 overall images [ 15.0 % for validation set]
250 batch-size,  0.025 learning rate,  1e-05 decay,  0.85 nesterov-momentum
Dropout-Rates: Layer 1-2: 2 %, Layer 1-2: 3 %, Layer 1-2: 20 %, Layer 1-2: 45 %
Epoch 1/15
96/95 [==============================] - 88s - loss: 0.0589 - val_loss: 0.0402
Epoch 2/15
96/95 [==============================] - 76s - loss: 0.0432 - val_loss: 0.0374
[...]
Epoch 13/15
96/95 [==============================] - 76s - loss: 0.0298 - val_loss: 0.0274
Epoch 14/15
96/95 [==============================] - 76s - loss: 0.0292 - val_loss: 0.0271
Epoch 15/15
96/95 [==============================] - 76s - loss: 0.0289 - val_loss: 0.0266

works better, before returning to this, learning rate and dropout rate was increased, resulting in worse losses after 15 epochs, so returned back to the parameters and ran 15 epochs. still no overfit, and parametrers could be evaluated for better parametrization, but task is fulfilled.